**texto en negrita**# Práctico 3: Aprendizaje Supervisado

## Mentoría *Sesgos Cognitivos en Razonamientos Lógicos*

**Fecha de entrega:** 11/09


Aplicaremos algoritmos de regresión de aprendizaje supervisado para predecir los índices de creencias en razonamientos lógicos y así detectar posibles sesgos.

- Comenzaremos utilizando un modelo base (baseline model) como referencia para comparar con otros modelos en la resolución de nuestro problema.

- Realizaremos optimización de hiperpárametros utilizando técnicas como grid search y random search.

- Seleccionaremos métricas de error como MAE, MSE y RMSE, para evaluar nuestros modelos

- Opcionalmente, veremos cómo transformar nuestro problema de regresión en un problema de clasificación binaria

------------------------------------------------------------------------------------------------------------------------

## 1. Selección de características y división en conjunto de entrenamiento y conjunto de prueba



In [ ]:
# importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

In [1]:
df1 = pd.read_csv("datos_preprocesados_grupo_1.csv")
df2 = pd.read_csv("datos_preprocesados.csv")
df1

NameError: ignored

### 1.1. Indicar qué características se utilizaran durante el entrenamiento

En el trabajo práctico anterior realizamos la curación de datos que creimos pertinente, obteniendo los datos arriba mostrados. A las columnas al estilo %_ValidezxCreencia se les aplicó una codificación ordinal, lo cuál puede ser que no haya sido la mejor decisión dado que no existe una relación jerárquica entre los distintos valores que tomaba esta variable. Al contar ahora con un nuevo dataframe preprocesado, se nos ocurre realizar el entrenamiento con ambos datasets, para evaluar la influencia de esta decisión en el entrenamiento de los modelos, y optar finalmente por el camino que de mejores resultados.<br>
Para el entrenamiento descartaremos las columnas relacionadas a los sujetos, como la aceptación y la corrección de los silogismos, dado que al estar estrechamente relacionado con la variable target, estaríamos overfitteando e introduciendo sesgos en nuestro propio análisis de sesgos.
Haremos una pequeña curación de datos al nuevo dataset obtenido y luego podremos alternar entre uno y otro.

In [ ]:
#Eliminar Aceptación y correctas
aceptacion = ['1_Aceptación', '2_Aceptación', '3_Aceptación', '4_Aceptación',
       '5_Aceptación', '6_Aceptación', '7_Aceptación', '8_Aceptación']
correctas = ['1_Correctas', '2_Correctas', '3_Correctas', '4_Correctas',
       '5_Correctas', '6_Correctas', '7_Correctas', '8_Correctas']

df1.drop(aceptacion+correctas, axis=1, inplace=True)

In [ ]:
df1.columns

Index(['Participante', 'Modalidad', '1_ValidezxCreencia', '2_ValidezxCreencia',
       '3_ValidezxCreencia', '4_ValidezxCreencia', '5_ValidezxCreencia',
       '6_ValidezxCreencia', '7_ValidezxCreencia', '8_ValidezxCreencia',
       'indice_creencia_norm'],
      dtype='object')

### 1.2. Utilizar esta sección para llevar a cabo cualquier acción que consideren necesaria para pasar a la división y el posterior entrenamiento

**Nota**: Recuerden insertar comentarios en el código indicando las acciones que llevan a cabo con su justificación

En este apartado trabajamos sobre el nuevo datadrame adquirido, realizando una pequeña curación para el trabajo del mismo. <br>
Primero transformamos los valores categóricos que toman las variables tipo "Validez_sil%" y "Creencia_sil%" por valores numéricos. La convención utilizada es que para un silogismo "Válido" o "Creíble", la columna toma un valor 1, en caso del silogismo ser "Inválido" o "Increíble", la columna toma valor 0. De esta forma hacemos un encoding que no jerarquiza ni prioriza un valor sobre otro, como puede llegar a entenderse en el manejo del primer data frame. <br>
Luego de esto, eliminamos las columnas "ValidezxCreencia_sil%", dado que además de contener valores categóricas, dicha información ya está contenida en otras columnas que son númericas, así que no perdemos información. Por último, eliminamos las variables edad, género y grupo por los mismos motivos que fueron eliminadas en el dataframe original.

In [ ]:
# Definición de la función que transforma los valores obtenidos del df a nuestra convención
def transform_codification(value):
    if value == 'V' or value == 'C':
        return 1
    elif value == 'I':
        return 0
    else:
        return value

# Aplica la transformación a las columnas de validez y creencia
validez_columns = ['Validez_sil1', 'Validez_sil2', 'Validez_sil3', 'Validez_sil4', 'Validez_sil5', 'Validez_sil6', 'Validez_sil7', 'Validez_sil8']
creencia_columns = ['Creencia_sil1', 'Creencia_sil2', 'Creencia_sil3', 'Creencia_sil4', 'Creencia_sil5', 'Creencia_sil6', 'Creencia_sil7', 'Creencia_sil8']

for col in validez_columns + creencia_columns:
    df2[col] = df2[col].apply(transform_codification)

# Elimina las columnas de validez x creencia
validezxcreencia_columns = ['ValidezxCreencia_sil1', 'ValidezxCreencia_sil2', 'ValidezxCreencia_sil3', 'ValidezxCreencia_sil4', 'ValidezxCreencia_sil5', 'ValidezxCreencia_sil6', 'ValidezxCreencia_sil7', 'ValidezxCreencia_sil8']
df2.drop(validezxcreencia_columns, axis=1, inplace=True)



In [2]:
df2.columns

NameError: ignored

In [ ]:
# Elimina las columnas extras que eliminamos en el análisis del práctico 2
drop_extras = [ 'Edad', 'Género', 'Grupo']
df2.drop(drop_extras, axis=1, inplace=True)

Como dijimos en el punto 1.2, eliminamos las variables relacionadas a 'Aceptación' y 'Correctas'

In [ ]:
aceptacion2 = ['Aceptación_sil1', 'Aceptación_sil2', 'Aceptación_sil3', 'Aceptación_sil4',
               'Aceptación_sil5', 'Aceptación_sil6', 'Aceptación_sil7', 'Aceptación_sil8']
correctas2 = ['Correctas_sil1', 'Correctas_sil2', 'Correctas_sil3', 'Correctas_sil4',
              'Correctas_sil5', 'Correctas_sil6', 'Correctas_sil7', 'Correctas_sil8']
df2.drop(aceptacion2+correctas2, axis=1, inplace=True)

In [3]:
df2.columns

NameError: ignored

## 1.3. Dividir en conjunto de entrenamiento y prueba

Ahora sí realizamos una copia de ambos dataframes en la variable df, así que para entrenar el modelo usamos la misma variable, con la diferencia que comentamos o descomentamos el df que vamos a utilizar. Luego de esto hacemos la selección de los atributos para el entrenamiento y la división del conjunto en entrenamiento y prueba

In [5]:
# Hago una copia del datadrame utilizado para el entrenamiento
#df = df1.copy()
df = df2.copy()

NameError: ignored

In [6]:
# Indicamos las características que se utilizarán (X) y la variable objetivo (y)
X = df.drop(['Participante', 'Modalidad', 'indice_creencia_norm'], axis=1)
y = df['indice_creencia_norm']

NameError: ignored

In [ ]:
X.columns

Index(['Validez_sil1', 'Validez_sil2', 'Validez_sil3', 'Validez_sil4',
       'Validez_sil5', 'Validez_sil6', 'Validez_sil7', 'Validez_sil8',
       'Creencia_sil1', 'Creencia_sil2', 'Creencia_sil3', 'Creencia_sil4',
       'Creencia_sil5', 'Creencia_sil6', 'Creencia_sil7', 'Creencia_sil8'],
      dtype='object')

In [7]:
# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

NameError: ignored

## 2. Modelo base

En cada proyecto, es útil crear un modelo de referencia que implemente un algoritmo muy simple. Esto nos permite comparar nuestros resultados posteriores con el modelo base y ver si estamos mejorando.
### 2.1.  Crear un modelo que siempre devuelva el índice de creencia promedio.

**Ayuda:** scikit-learn cuenta con la clase [DummyRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html) que es muy útil para esta tarea.
_______________

In [ ]:
# Crear un modelo DummyRegressor que prediga siempre el índice de creencia promedio
dummy_model = DummyRegressor(strategy='mean')

# Entrenar el modelo
dummy_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_dummy = dummy_model.predict(X_test)

# Calcular métricas de error
mae_dummy = mean_absolute_error(y_test, y_pred_dummy)
mse_dummy = mean_squared_error(y_test, y_pred_dummy)
rmse_dummy = np.sqrt(mse_dummy)
score_dummy = dummy_model.score(X_test, y_test)

print("Modelo Dummy - Accuracy:", score_dummy)
print("Modelo Dummy - MAE:", mae_dummy)
print("Modelo Dummy - MSE:", mse_dummy)
print("Modelo Dummy - RMSE:", rmse_dummy)

Modelo Dummy - Accuracy: -0.0022405646004968194
Modelo Dummy - MAE: 0.2254344919786096
Modelo Dummy - MSE: 0.0809511638880151
Modelo Dummy - RMSE: 0.2845191801759859


### 2.2 Evaluación
Una vez que hemos entrenado nuestro modelo base y obtenido predicciones para nuestro conjunto de test, es hora de que evaluamos su performance. Para la evaluación usaremos el [error absoluto medio](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) (MAE por sus siglas en inglés).

Adicionalmente, van a tener que elegir otra métrica de error y justificar su elección.
____________________

Como métrica adicional utilizaremos el WMAE (error absoluto medio ponderado). El WMAE es similar al MAE (Mean Absolute Error), pero tiene en cuenta el peso de cada observación. El peso de cada observación se puede utilizar para reflejar la importancia relativa de cada observación. En este caso, estamos asignando un peso de 2 a las observaciones que son mayores que 0, y un peso de 0.5 a las observaciones que son menores que 0. Esto significa que el modelo le dará más importancia a las observaciones que son mayores que 0, dado que estas nos indican presencia de sesgos en nuestras observaciones, y menos importancia a las observaciones que son menores que 0 (que como vimos en el práctico anterior, los valores negativos de la variable target no tenía mucho interés de estudio). El resultado será un número que indica el error del modelo en la predicción de las observaciones, teniendo en cuenta los pesos de las observaciones. Un WMAE más bajo indica que el modelo es más preciso.

In [ ]:
mae_mod_base = float("{:.2f}".format(mae_dummy))

# Calcular los pesos
weights = []
for y_i in y_test:
    if y_i > 0:
        weights.append(2)
    else:
        weights.append(0.5)

# Calcular el error del modelo con MAE
wmae = mean_absolute_error(y_test, y_pred_dummy, sample_weight=weights)

print("Modelo Dummy - WMAE:", wmae)

Modelo Dummy - WMAE: 0.19531840259039393


Antes de pasar a la siguiente sección vamos a llevar a cabo un pequeño test para ver que nuestro modelo base no sobrepase el valor máximo para el MAE que fijamos en $0.24$

In [ ]:
# no modificar esta celda

def check_mae(mae):
    if mae <= 0.24:
        print(f"MAE:{mae}")
    else:
        raise ValueError(f'El MAE es de {mae}, necesitan un MAE menor o igual que 0.24. Revisen las características seleccionadas')

In [ ]:
check_mae(mae_mod_base)

MAE:0.23


## 2. Experimentos

### 2.1. Probando diferentes estimadores

Utilice tres estimadores diferentes para la predicción del índice de creencia, obtenga las predicciones y realice la evaluación

**Nota:** Pueden probar la cantidad de modelos que deseen, pero aquí en la notebook deben registrar sólo tres.


In [ ]:
# Primer estimador
estimador_1 = GradientBoostingRegressor(n_estimators=5, max_depth=3, random_state=42)

# train
estimador_1.fit(X_train, y_train)

# predicciones
y_pred_1 = estimador_1.predict(X_test)

# evaluación
mae_1 = mean_absolute_error(y_test, y_pred_1)
weights_1 = []
for y_i in y_test:
    if y_i > 0:
        weights_1.append(2)
    else:
        weights_1.append(0.5)
wmae_1 = mean_absolute_error(y_test, y_pred_1, sample_weight=weights_1)
print("MAE:", mae_1)
print("WMAE:", wmae_1)

MAE: 0.2254344919786096
WMAE: 0.19531840259039393


In [ ]:
# Segundo estimador
estimator_2 = RandomForestRegressor(random_state=42)

# train
estimator_2.fit(X_train, y_train)

# predicciones
y_pred_2 = estimator_2.predict(X_test)

# evaluación
mae_2 = mean_absolute_error(y_test, y_pred_2)
weights_2 = []
for y_i in y_test:
    if y_i > 0:
        weights_2.append(2)
    else:
        weights_2.append(0.5)
wmae_2 = mean_absolute_error(y_test, y_pred_2, sample_weight=weights_2)
print("MAE:", mae_2)
print("WMAE:", wmae_2)

MAE: 0.22490808823529412
WMAE: 0.19528197517539125


In [ ]:
# Tercer estimador: KNeighborsRegressor
estimador_3 = KNeighborsRegressor(n_neighbors=5)

# train
estimador_3.fit(X_train, y_train)

# Predicciones
y_pred_3 = estimador_3.predict(X_test)

# evaluación
mae_3 = mean_absolute_error(y_test, y_pred_3)
weights_3 = []
for y_i in y_test:
    if y_i > 0:
        weights_3.append(2)
    else:
        weights_3.append(0.5)
wmae_3 = mean_absolute_error(y_test, y_pred_3, sample_weight=weights_3)
print("MAE 3:", mae_3)
print("WMAE 3:", wmae_3)

MAE 3: 0.27982954545454547
WMAE 3: 0.19908256880733946


### 2.2. Optimización de hiperparámetros


Seleccione un estimador de los utilizados en el punto anterior y lleve a cabo una optimización de hiperparámetros utilizando  [Grid Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) o [Random Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

Registre las mejores métricas alcanzadas y los valores de los hiperparámetros utilizados.

In [ ]:
estimator_2 = RandomForestRegressor(random_state=42)

# Hiperparámetros a ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(estimator_2, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y la mejor puntuación
best_params = grid_search.best_params_
best_mae = -grid_search.best_score_

# Modelo con los mejores hiperparámetros
estimator_opt = RandomForestRegressor(**best_params, random_state=42)
estimator_opt.fit(X_train, y_train)

# predicciones
y_pred = estimator_opt.predict(X_test)

# evaluacion
mae_est_opt = mean_absolute_error(y_test, y_pred)
wmae_est_opt = mean_absolute_error(y_test, y_pred, sample_weight=weights)

# Imprimir los resultados
print("Mejores hiperparámetros:", best_params)
print("Mejor MAE:", best_mae)
print("MAE del modelo optimizado:", mae_est_opt)
print("WMAE del modelo optimizado:", wmae_est_opt)

Mejores hiperparámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Mejor MAE: 0.23547458842214422
MAE del modelo optimizado: 0.2244986631016043
WMAE del modelo optimizado: 0.19525364274150028


In [ ]:
estimador = KNeighborsRegressor()
param_dist = {
    'n_neighbors': np.arange(1, 30),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Crear un objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimador, param_distributions=param_dist, n_iter=30, cv=10, scoring='neg_mean_absolute_error')

# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
random_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y la mejor puntuación
best_params = random_search.best_params_
best_mae = -random_search.best_score_

# Crear un modelo con los mejores hiperparámetros encontrados
estimador_3_opt = KNeighborsRegressor(**best_params)

# Entrenamiento con los mejores hiperparámetros
estimador_3_opt.fit(X_train, y_train)

# Predicciones con el modelo optimizado
y_pred_3_opt = estimador_3_opt.predict(X_test)

# Evaluación con MAE
mae_3_opt = mean_absolute_error(y_test, y_pred_3_opt)
weights_3_opt = []
for y_i in y_test:
    if y_i > 0:
        weights_3_opt.append(2)
    else:
        weights_3_opt.append(0.5)
wmae_3_opt = mean_absolute_error(y_test, y_pred_3_opt, sample_weight=weights_3_opt)

#Imprimo resultados
print("Mejores hiperparámetros:", best_params)
print("Mejor MAE:", best_mae)
print("MAE del modelo optimizado:", mae_3_opt)
print("WMAE del modelo optimizado:", wmae_3_opt)

ValueError: 
All the 300 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\lisai\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\lisai\miniconda3\lib\site-packages\sklearn\neighbors\_regression.py", line 217, in fit
    return self._fit(X, y)
  File "C:\Users\lisai\miniconda3\lib\site-packages\sklearn\neighbors\_base.py", line 454, in _fit
    X, y = self._validate_data(
  File "C:\Users\lisai\miniconda3\lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\lisai\miniconda3\lib\site-packages\sklearn\utils\valida

In [ ]:
check_mae(mae_3_opt)

MAE:0.22017045454545456


##### Conclusiones
El mejor modelo que pudimos entrenar es el KNeighborsRegressor, obteniendo un MAE de 0.22017 y un WMAE de 0.19495, frente a los scores obtenidos con el modelo dummy (MAE: 0.22543 y WMAE: 0.19532).

## 3. Ingeniería de características y re-entrenamiento del modelo

Como ya habrán podido observar a lo largo de la diplomatura, mucho de los procesos en ciencia de datos son iterativos. Cuando entrenamos modelos, esto implica agregar y eliminar características, modificar el escalado y la codificación, y otros tipos de acciones que nos permitan mejorar la performance de nuestro modelo.

Aquí están algunas acciones que pueden llevar a cabo para mejorar el rendimiento del modelo:
<br></br>
- Agregar nuevas características a los datos. Esto puede ayudar al modelo a aprender más sobre los datos y a hacer mejores predicciones.

- Eliminar características irrelevantes de los datos. Esto puede ayudar al modelo a evitar el sobreajuste y a mejorar su generalización.

- Modificar el escalado de los datos. Esto puede ayudar al modelo a aprender más rápido y a hacer mejores predicciones.

- Modificar la codificación de los datos. Esto puede ayudar al modelo a entender mejor los datos y a hacer mejores predicciones.


<br></br>
Una vez que hayan realizado alguna/s de estas acciones, deben reentrenar el modelo utilizado en el punto anterior.

Finalmente, recuerden registrar las métricas de error, esto les ayudará a determinar si con estas acciones han mejorado el rendimiento del modelo.
__________________________

Cargamos nuevamente el dataset n°2, que fue el que nos dio mejores resultados. En esta instancia, nos quedaremos con las mismas features elegidas pero agregaremos también la columna de 'Modalidad'. El problema es que esta columna contiene datos categóricos sobre la modalidad de resolución de la actividad, por lo que debemos hacer una codificación para que el modelo pueda entrenar con esta columna extra.

In [ ]:
#Cargamos los datos y eliminamos atributos previamente descartados, a excepción de modalidad
df = pd.read_csv("datos_preprocesados.csv")

df.drop(validezxcreencia_columns, axis=1, inplace=True)
df.drop(drop_extras, axis=1, inplace=True)
df.drop(aceptacion2+correctas2, axis=1, inplace=True)
df.drop('Participante', axis=1, inplace=True)
df.columns

Index(['Modalidad', 'Validez_sil1', 'Validez_sil2', 'Validez_sil3',
       'Validez_sil4', 'Validez_sil5', 'Validez_sil6', 'Validez_sil7',
       'Validez_sil8', 'Creencia_sil1', 'Creencia_sil2', 'Creencia_sil3',
       'Creencia_sil4', 'Creencia_sil5', 'Creencia_sil6', 'Creencia_sil7',
       'Creencia_sil8', 'indice_creencia_norm'],
      dtype='object')

In [ ]:
#Hacemos la división en train y test, para luego hacer el OHE
X = df.drop(['indice_creencia_norm'], axis=1)
y = df['indice_creencia_norm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [8]:
#Verificamos la cantidad de filas y columnas
print(f"""Conjunto de train: {X_train.shape[0]} filas y {X_train.shape[1]} columnas.
Conjunto de test: {X_test.shape[0]} filas y {X_test.shape[1]} columnas. """)

NameError: ignored

In [ ]:
print(*X['Modalidad'].unique(), sep='\n')

Resolución Individual
Resolución Grupal
Grupal entre sujetos
Resolución Individual Post
Resolución Individual Pre


In [9]:
#Realizamos el encoder para cada set
ohe_enc = OneHotEncoder(sparse_output=False)
mod_enc_train = ohe_enc.fit_transform(X_train[['Modalidad']])
mod_enc_test = ohe_enc.transform(X_test[['Modalidad']])

NameError: ignored

In [10]:
print(mod_enc_train.shape)
print(mod_enc_test.shape)

NameError: ignored

In [11]:
#Reseteamos el índice y concatenamos con el df anterior, sin la columna de modalidad ya que dicha columna se encuentra codeada ya
X_train_encoded = pd.concat([X_train.drop('Modalidad', axis=1).reset_index(drop=True),
                            pd.DataFrame(mod_enc_train)], axis=1)

X_test_encoded = pd.concat([X_test.drop('Modalidad', axis=1).reset_index(drop=True),
                           pd.DataFrame(mod_enc_test)], axis=1)

NameError: ignored

In [ ]:
#Verificamos que tengamos la cantidad de filas y columnas correctas
print(f"""Conjunto de train con columna categórica codificada: {X_train_encoded.shape[0]} filas y {X_train_encoded.shape[1]} columnas.
Conjunto de test con columna categórica codificada: {X_test_encoded.shape[0]} filas y {X_test_encoded.shape[1]} columnas. """)

Conjunto de train con columna categórica codificada: 408 filas y 21 columnas.
Conjunto de test con columna categórica codificada: 176 filas y 21 columnas. 


Eran 5 categorías y eliminamos la variable original ('Modalidad') por lo cual deberían haberse agregado 4 columnas (5-1). Como vemos efectivamente es así, además el número de filas se mantuvo intacto, como debe ser.

In [12]:
#Le ponemos las etiquetas correctas a las variables codeadas
X_train_enc_2 = pd.concat([X_train.reset_index(drop=True),
                              pd.DataFrame(mod_enc_train, columns = ohe_enc.categories_[0])],  axis = 1)
X_test_enc_2 = pd.concat([X_test.reset_index(drop=True),
                              pd.DataFrame(mod_enc_test, columns = ohe_enc.categories_[0])],  axis = 1)

X_train_enc_2.drop('Modalidad', axis=1, inplace=True)
X_test_enc_2.drop('Modalidad', axis=1, inplace=True)

NameError: ignored

In [ ]:
#Realizamos la codificación de las columnas validez y creencia
for col in validez_columns + creencia_columns:
    X_train_enc_2[col] = X_train_enc_2[col].apply(transform_codification)

for col in validez_columns + creencia_columns:
    X_test_enc_2[col] = X_test_enc_2[col].apply(transform_codification)

In [ ]:
X_train_enc_2

,Validez_sil1,Validez_sil2,Validez_sil3,Validez_sil4,Validez_sil5,Validez_sil6,Validez_sil7,Validez_sil8,Creencia_sil1,Creencia_sil2,...,Creencia_sil4,Creencia_sil5,Creencia_sil6,Creencia_sil7,Creencia_sil8,Grupal entre sujetos,Resolución Grupal,Resolución Individual,Resolución Individual Post,Resolución Individual Pre
0,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,0.0,0.0,1.0,0.0
1,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,1.0,0.0,0.0,0.0
2,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,1.0,0.0,0.0,0.0
3,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,1.0,0.0,0.0,0.0
4,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,0.0,1.0,0.0,0.0
404,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,1.0,0.0,0.0,0.0
405,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,0.0,0.0,0.0,1.0
406,1,1,0,0,1,1,0,0,1,0,...,0,1,0,1,0,0.0,0.0,1.0,0.0,0.0


Ahora sí, probamos el mejor modelo obtenido (KNeighborsRegressor) con una

---

optimización de hiperparámetros, para ver si el rendimiento mejora con las nuevas columnas incorporadas.

In [ ]:
estimador_3 = KNeighborsRegressor(n_neighbors=3)

# train
estimador_3.fit(X_train_enc_2, y_train)

# Predicciones
y_pred_3 = estimador_3.predict(X_test_enc_2)

# evaluación
mae_3 = mean_absolute_error(y_test, y_pred_3)
weights_3 = []
for y_i in y_test:
    if y_i > 0:
        weights_3.append(2)
    else:
        weights_3.append(0.5)
wmae_3 = mean_absolute_error(y_test, y_pred_3, sample_weight=weights_3)
print("MAE 3:", mae_3)
print("WMAE 3:", wmae_3)

MAE 3: 0.28740530303030304
WMAE 3: 0.26620795107033635


In [ ]:
estimador = KNeighborsRegressor()
param_dist = {
    'n_neighbors': np.arange(1, 30),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Crear un objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimador, param_distributions=param_dist, n_iter=30, cv=10, scoring='neg_mean_absolute_error', random_state=42)

# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
random_search.fit(X_train_enc_2, y_train)

# Obtener los mejores hiperparámetros y la mejor puntuación
best_params = random_search.best_params_
best_mae = -random_search.best_score_

# Crear un modelo con los mejores hiperparámetros encontrados
estimador_3_opt = KNeighborsRegressor(**best_params)

# Entrenamiento con los mejores hiperparámetros
estimador_3_opt.fit(X_train_enc_2, y_train)

# Predicciones con el modelo optimizado
y_pred_3_opt = estimador_3_opt.predict(X_test_enc_2)

# Evaluación
mae_3_opt = mean_absolute_error(y_test, y_pred_3_opt)
weights_3_opt = []
for y_i in y_test:
    if y_i > 0:
        weights_3_opt.append(2)
    else:
        weights_3_opt.append(0.5)
wmae_3_opt = mean_absolute_error(y_test, y_pred_3_opt, sample_weight=weights_3_opt)

#Imprimimos resultados
print("Mejores hiperparámetros:", best_params)
print("Mejor MAE:", best_mae)
print("MAE del modelo optimizado:", mae_3_opt)
print("WMAE del modelo optimizado:", wmae_3_opt)

Mejores hiperparámetros: {'weights': 'uniform', 'p': 2, 'n_neighbors': 18}
Mejor MAE: 0.23727981029810294
MAE del modelo optimizado: 0.2267992424242424
WMAE del modelo optimizado: 0.19877675840978593


### 3.1. Evaluación

Atendiendo a lo realizado hasta ahora, responda las siguientes preguntas:
- ¿Observan diferencias en el rendimiento del modelo base con el que empezaron y el de los diferentes modelos entrenados posteriormente?

- ¿Hubo diferencias en el rendimiento del modelo con parámetros optimizados antes y después de trabajar con las características?¿Por qué considera que sucede esto?
_____________________

Con el mejor modelo logramos alcanzar un MAE de 0.2202, en contraste con el MAE de 0.2254 obtenido utilizando el modelo dummy. Aunque esta mejora puede parecer modesta a primera vista, consideremos que nuestra variable objetivo se encuentra en el rango de -1 a 1, y en su mayoría toma valores en el intervalo de 0 a 1 (valores pequeños), por lo que esta diferencia es apreciable.

Destacamos que el rendimiento de los modelos mejoró significativamente con el conjunto de datos preprocesado adquirido en comparación al dataset preprocesado de trabajos anteriores. Por este motivo, observamos que la decisión de utilizar una codificación ordinal para la variable 'ValidezxCreencia' no fue la más acertada.

En cuanto a la inclusión de la variable 'Modalidad', no obtuvimos mejoras significativas. Es posible que esto se deba a la codificación utilizada previamente para las columnas 'Validez' y 'Creencia'. Consideramos que una opción más efectiva podría ser aplicar la codificación 'One-Hot Encoding' (OHE) para cada una de estas columnas. Por ejemplo, en lugar de tener 'Validez_sil1' y 'Creencia_sil1' como columnas individuales, podríamos crear columnas binarias separadas, como 'Validez_sil1_V', 'Validez_sil1_I', 'Creencia_sil1_C', 'Creencia_sil1_I', y así sucesivamente para cada silogismo.

## 4. (Opcional) Replanteando nuestro problema

A lo largo de esta actividad hemos abordado el problema de predecir el índice de creencia. Este índice es muy útil para observar la polarización en las respuestas y analizar aspectos más sútiles de la problemática.

Sin embargo, podríamos simplemente querer predecir si tienen lugar o no dichos sesgos, sin importar su grado, con lo cual podríamos replantear nuestro problema como un problema de clasificación binaria. En este tipo de problemas, el objetivo es predecir si una instancia pertenece a una clase o a otra. En nuestro caso, las dos clases son "presencia de sesgos" y "ausencia de sesgos".


Para hacer esto, podemos modificar nuestra columna con la variable objetivo. Los valores positivos se reemplazarán con la etiqueta $1$, que indicarán la presencia de sesgos. Mientras que, los valores iguales o menores que $0$ se reemplazarán con la etiqueta $0$, que indicarán la ausencia de sesgos.

Como ejercicio opcional, los invitamos a que entrenen un modelo que emplee algoritmos de clasificación para predecir la ausencia o presencia de sesgos de creencia.

## 5. Informe

*Elaboren* un breve informe de lo realizado durante esta actividad práctica reseñando aspectos salientes, dificultades encontradas, etc.